### Dispatcher Servlet
Idea is to act as single point for all request. To configure dispatcher servlet, in web.xml:
```xml
<servlet>
    <servlet-name>app</servlet-name>
    <servlet-class>org.springframework.web.servlet.DispatcherServlet</servlet-class>
    <load-on-startup>1</load-on-startup>
</servlet>
```

Dispatcher servlet needs a `WebApplicationContext`. By default it will search for `<servlet-name>-context.xml` file in WEB-INF folder for bean definition.

The other way is to use Java class for defining Dispatcher servlet. In the below example, we will use a Java based configuration (instead of XML)

In [ ]:
public class MyWebApplicationInitializer implements WebApplicationInitializer {

    @Override
    public void onStartup(ServletContext servletContext) {

        // Load Spring web application configuration
        AnnotationConfigWebApplicationContext context = new AnnotationConfigWebApplicationContext();
        context.register(AppConfig.class);

        // Create and register the DispatcherServlet
        DispatcherServlet servlet = new DispatcherServlet(context);
        ServletRegistration.Dynamic registration = servletContext.addServlet("app", servlet);
        registration.setLoadOnStartup(1);
        registration.addMapping("/");
    }
}

The configuration file:

In [ ]:
@Configuration
public class AppConfig {
}

### Dispatcher Servlet Initialization
As the dispatcher servlet starts it looks into the WebApplicationContext for some special beans. If it doesn't find those beans it uses the default beans of each type. The special beans it looks for are:
- `MultipartResolver` : defaults to none
- `LocaleResolver`: defaults to `AcceptHeaderLocaleResolver`
- `ThemeResolver`: defaults to `FixedThemeResolver`
- `ViewResolver`s: decides what `View` to use to render response. Defaults to `InternalResourceViewResolver`
- `HandlerMapping`s: maps a request to a handler. Defaults to list `BeanNameUrlHandler`, `RequestMappingHandlerMapping`, `RouterFunctionMapping`
- `HandlerAdapter`s: invokes a handler mapped to a request. Defaults to list `SimpleControllerHandlerAdapter`, `HttpRequestHandlerAdapter`, `RequestMappingHandlerAdapter`, `HandlerFunctionAdapter`
- `HandlerExceptionResolver`s: defaults to none

If we do not want to use the defaults, we can configure a corresponding bean and Dispatcher servlet will use that instead. For example if we want a ViewResolver to use JSPs in a specific folder:

In [ ]:
@Configuration
public class AppConfig {
    @Bean
    public InternalResourceViewResolver resolver() {
        InternalResourceViewResolver resolver = new InternalResourceViewResolver();
        // The default version doesn't have the below two
        // properties set
        resolver.setPrefix("/WEB-INF/views/");
        resolver.setSuffix(".jsp");
        return resolver;
    }
}

### @EnableWebMvc
We can add the above annotation to our configuration file to make available some useful beans. The most important being `HttpMessageConverter`. It also enable support for validators and formatters.

In [ ]:
@EnableWebMvc
@Configuration
public class AppConfig {
}

The XML equivalent is:
```xml
<?xml version="1.0" encoding="UTF-8"?>
<beans xmlns="http://www.springframework.org/schema/beans"
    xmlns:mvc="http://www.springframework.org/schema/mvc"
    xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
    xsi:schemaLocation="
        http://www.springframework.org/schema/beans
        https://www.springframework.org/schema/beans/spring-beans.xsd
        http://www.springframework.org/schema/mvc
        https://www.springframework.org/schema/mvc/spring-mvc.xsd">

    <!--Equivalent-->
    <mvc:annotation-driven/>

</beans>
```

### Dispatcher Servlet Flow
![DS Flow](https://i.imgur.com/C4keNkJ.png)